In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests

In [15]:

def get_reviews(soup):
    reviews = []
    try:
        review_tags = soup.find_all('div',attrs={'class':'a-expander-content reviewText review-text-content a-expander-partial-collapse-content'})
        
        for review_tag in review_tags:
            reviews.append(review_tag.text.strip())
    except AttributeError:
        pass
    return reviews


def get_user_names(soup):
    user_names = []
    try:
        user_tags = soup.find_all('span',attrs={'class':'a-profile-name'})
        for user_tag in user_tags:
            user_names.append(user_tag)
    except AttributeError:
        pass
    return user_names

In [16]:
if __name__ == '__main__':

    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15',
        'Accept-Language': 'en-US,en;q=0.5'
    }

    # The webpage URL
    URL = "https://www.amazon.com/s?k=macbook&crid=39RUATP4XMNJ4&sprefix=macbo%2Caps%2C396&ref=nb_sb_ss_ts-doa-p_1_5"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"user_reviews":[]}

    # Loop for extracting product details from each link
    for link in links_list:
        if not link.startswith('http'):
            link = "https://www.amazon.com" + link
        new_webpage = requests.get(link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        d['user_reviews'].append(get_reviews(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df.to_csv('../Scraped_datasets/review_list.csv', header=True, index=False)

In [17]:
amazon_df

,user_reviews
0,"[Definitely not an iPhone..fast, light weight ..."
1,[Love how slim and light weight this is. If yo...
2,[Windows user here. Got tired of 1 to 2 hours ...
3,[Works great with Lightroom Classic and large ...
4,"[Great Product and amazing service!, Macbook P..."
5,[Summary: I just did my new music album with t...
6,[The Apple 2023 MacBook Air is a stellar examp...
7,[I have used Mac products for over 30 years. T...
8,[The service from this seller on Amazon was go...
9,[]


In [18]:
amazon_df.iloc[4]

user_reviews    [Great Product and amazing service!, Macbook P...
Name: 4, dtype: object

In [19]:
amazon_df['user_reviews'][0]

['Definitely not an iPhone..fast, light weight all together beautiful machine',
 "The 13.6-inch Liquid Retina display delivers stunning visuals with vibrant colors and sharp details. The M3 chip offers blazing-fast performance, ensuring seamless multitasking and smooth operation. With 8GB of unified memory and 256GB SSD storage, it provides ample space for all your needs. The backlit keyboard and Touch ID add convenience and security, while the 1080p FaceTime HD camera ensures crystal-clear video calls. In Midnight color, it's not just a laptop; it's a statement of style and sophistication.",
 "I use my laptop for work purposes, connecting to all sorts of external multimedia equipment. I had a MacBook Pro for nearly a decade, and it was more than showing its great age. I knew I needed to upgrade, but I was worried about choosing the Air. I liked the features, I liked the slender form and low weight. I just worried it would not do what I needed it to do.I needn't have worried. This is b

In [21]:
from textblob import TextBlob
# Calculate sentiment for each review
sentiments = [TextBlob(review).sentiment.polarity for review in amazon_df['user_reviews'][0]]

In [22]:
sentiments

[0.4166666666666667,
 0.19027777777777777,
 0.34027777777777785,
 -0.06833333333333333,
 0.07100108225108226,
 0.4375,
 1.0,
 0.0,
 0.3444444444444444,
 -0.05,
 0.034999999999999976]

In [24]:
sum(sentiments)/len(sentiments)

0.24698494687131056